In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# Proxy configuration
PROXY = "127.0.0.1:24000"
# Path to chromedriver executable
PATH = "/opt/homebrew/Caskroom/chromedriver/121.0.6167.85/chromedriver-mac-arm64/chromedriver"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

# Initialize Chrome WebDriver with the specified path and options
service = Service(PATH)
# chrome = webdriver.Chrome(service=service, options=chrome_options)

# player_ids = ['677950', '572233', '682998', '666150', '656896', '553993', '672515', '672695', '607054', '664983', '645444', '606466', '666971', '656976', '592626']
timeout = 25
mlb_seasons = ['2023', '2022', '2021']
player_ids = ['677950']

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [ ]:
url = r"https://baseballsavant.mlb.com/visuals/statcast-pitch-highlighter?playerId=502671&type=&batHand=&pitchHand=&pitchType=&year=2023&zone="
driver.get(url)

element_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="ddlPitchTypes"]'))
WebDriverWait(driver, timeout).until(element_present)

headers_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="calculations"]/table/thead/tr'))
WebDriverWait(driver, timeout).until(headers_present)

select = Select(driver.find_element(By.XPATH, r'//*[@id="ddlPitchTypes"]'))
select.select_by_index(0)
pitchType = select.first_selected_option.text
src = driver.page_source
parser = BeautifulSoup(src, "lxml")
table = parser.find("div", attrs = {"id": "calculations"})
headers = table.findAll('th')
headerlist = [h.text.strip() for h in headers[:]]
headerlist[0] = "Name"

stats = pd.DataFrame(columns=headerlist)

In [ ]:
for player in player_ids:
    for season in mlb_seasons:
        for i in range(len(select.options)):
            url = f"https://baseballsavant.mlb.com/visuals/statcast-pitch-highlighter?playerId={player}&type=&batHand=&pitchHand=&pitchType=&year={season}&zone="
            driver.get(url)

            element_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="ddlPitchTypes"]'))
            WebDriverWait(driver, timeout).until(element_present)

            data_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="calculations-tr_0"]'))
            WebDriverWait(driver, timeout).until(data_present)
            
            select = Select(driver.find_element(By.XPATH, r'//*[@id="ddlPitchTypes"]'))
            select.select_by_index(f"{i}")

            pitches_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="viz"]/div[2]/h6/h6'))
            WebDriverWait(driver, timeout).until(pitches_present)
            pitches_message = driver.find_element(By.XPATH, r'//*[@id="viz"]/div[2]/h6/h6').text
            if pitches_message == 'No Data Found':
                break
                
            pitchType = select.first_selected_option.text
            src = driver.page_source
            parser = BeautifulSoup(src, "lxml")
            table = parser.find("div", attrs = {"id": "calculations"})
            headers = table.findAll('th')
            headerlist = [h.text.strip() for h in headers[:]]
            headerlist[0] = "Name"
            rows = table.findAll('tr')[1:]
            player_stats = [[td.getText().strip() for td in rows[i].findAll('td')] for i in range(len(rows))]
            if player_stats[0][4] == "": player_stats[0][4] = pitchType
            if player_stats[0][1] == "": player_stats[0][1] = season

            stats2 = pd.DataFrame(player_stats, columns=headerlist)
            stats = pd.concat([stats,stats2], ignore_index=True)

driver.quit()

In [ ]:
stats

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# Proxy configuration
PROXY = "127.0.0.1:24000"
# Path to chromedriver executable
PATH = "/opt/homebrew/Caskroom/chromedriver/121.0.6167.85/chromedriver-mac-arm64/chromedriver"

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

# Initialize Chrome WebDriver with the specified path and options
service = Service(PATH)

timeout = 25
teams = ["dbacks", "braves", "orioles", "redsox", "cubs", "whitesox", "reds", "guardians", "rockies", "tigers", "astros", "royals", "angels", "dodgers", "marlins", "brewers", "twins", "mets", "yankees", "athletics", "phillies", "rays", "rangers", "bluejays", "rangers", "nationals"]
teams = ["dbacks"]

players = []
position = ''

In [3]:
# Setting up WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [4]:
for team in teams:
    # Accessing the URL
    url = f"https://www.mlb.com/{team}/roster"
    driver.get(url)

    # Getting the page source
    src = driver.page_source
    parser = BeautifulSoup(src, "lxml")

    # Finding the table containing players
    table = parser.find("div", attrs = {"class": "players"})

    # Getting all 'tr' elements, excluding the first one 
    rows = table.findAll('tr')[:]
    
    # Extracting player information
    for row in rows:
        positionI = row.find('td', attrs = {"colspan": '2'})
        if positionI:
            position = positionI.get_text()
            position = position[:len(position)-1]
            
        player_info = []
        for a in row.find_all('a'):
            player_info.append(team)
            player_info.append(a.getText().strip())
            player_info.append(a.attrs.get('href', 'Not found'))
            player_info.append(position)
        if player_info !=  []:
            players.append(player_info)
    

In [5]:
driver.quit()

In [6]:
players

[['dbacks', 'Miguel Castro', '/player/miguel-castro-612434', 'Pitcher'],
 ['dbacks', 'Slade Cecconi', '/player/slade-cecconi-677944', 'Pitcher'],
 ['dbacks', 'Luis Frías', '/player/luis-frías-666818', 'Pitcher'],
 ['dbacks', 'Zac Gallen', '/player/zac-gallen-668678', 'Pitcher'],
 ['dbacks', 'Kevin Ginkel', '/player/kevin-ginkel-656464', 'Pitcher'],
 ['dbacks', 'Tommy Henry', '/player/tommy-henry-674072', 'Pitcher'],
 ['dbacks', 'Drey Jameson', '/player/drey-jameson-686753', 'Pitcher'],
 ['dbacks', 'Bryce Jarvis', '/player/bryce-jarvis-686826', 'Pitcher'],
 ['dbacks', 'Merrill Kelly', '/player/merrill-kelly-518876', 'Pitcher'],
 ['dbacks', 'Joe Mantiply', '/player/joe-mantiply-573009', 'Pitcher'],
 ['dbacks', 'Corbin Martin', '/player/corbin-martin-656686', 'Pitcher'],
 ['dbacks', 'Justin Martinez', '/player/justin-martinez-679885', 'Pitcher'],
 ['dbacks', 'Scott McGough', '/player/scott-mcgough-543518', 'Pitcher'],
 ['dbacks', 'Ryne Nelson', '/player/ryne-nelson-669194', 'Pitcher'],
 [

In [ ]:
for row in rows:
    positionI = row.find('td', attrs = {"colspan": '2'})
    # print(positionI)
    if positionI:
        position = positionI.get_text()
        position = position[:len(position)-1]
    

In [7]:
position

'Outfielder'